In [1]:
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

%matplotlib inline

ox.config(log_console=True, use_cache=True)
weight_by_length = False

ox.__version__

'0.15.1'

In [6]:
path = '/Users/andrewrenninger/Desktop/R/git/networks/data/'
os.chdir(path)

In [8]:
camps = pd.read_csv("camps/camps.csv")

In [19]:
camps.head()

,X,Y,name
0,-6.865540,26.831305,Dakhla Refugee Camp
1,-8.010650,27.513829,Boujdour Refugee Camp
2,-8.023589,27.739910,El Aaiún Refugee Camp
3,-8.720793,7.025102,Bahn Refugee Camp
4,40.286872,0.190007,Dagahaley Refugee Camp


In [42]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [51]:
bearings = {}

for i in np.arange(1, camps.shape[0]):
    print(datetime.datetime.now(), camps.name[i])
    
    name = camps.name[i]
    query = (camps.Y[i], camps.X[i])
    
    try: 
        G = ox.graph_from_point(query, dist = 1000)
    except:
        pass
    
   # if G.edges < 100:
   #     pass
   # 
   # else:
    
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[name] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[i] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

2020-06-09 16:28:59.057337 Boujdour Refugee Camp
2020-06-09 16:29:00.441091 El Aaiún Refugee Camp
2020-06-09 16:29:02.894725 Bahn Refugee Camp
2020-06-09 16:29:03.950045 Dagahaley Refugee Camp
2020-06-09 16:29:07.371236 Melkadida Refugee Camp
2020-06-09 16:29:11.800669 Heleweyen Refugee Camp
2020-06-09 16:29:14.633696 Kobe Refugee Settlement
2020-06-09 16:29:20.437735 Bokolmanyo Refugee Camp
2020-06-09 16:29:25.887929 Hagadera Refugee Camp
2020-06-09 16:29:29.236806 Ifo 2 Refugee Camp
2020-06-09 16:29:33.743239 Buramino Refugee Settlement
2020-06-09 16:29:35.201922 Ampain
2020-06-09 16:29:36.031703 Boynuyoğun
2020-06-09 16:29:36.749152 Camp de réfugiés de Gondje مخيم قونجي
2020-06-09 16:29:39.253609 Amboko Refugee Camp
2020-06-09 16:29:40.338015 Al Zaatari Refugee Camp
2020-06-09 16:29:42.576325 Shagarab-III Refugee Camp
2020-06-09 16:29:43.911469 Shagarab-I Refugee Camp
2020-06-09 16:29:45.330616 Shagarab-II Refugee Camp
2020-06-09 16:29:46.680054 Kouankan
2020-06-09 16:29:48.107755 B

2020-06-09 16:40:58.186739 Motor Park Idp Camp Refugee Site
2020-06-09 16:41:02.425024 Masarmari Refugee Site
2020-06-09 16:41:05.485553 1000 Camp Dikwa Refugee Site
2020-06-09 16:41:08.799394 Ministry Of Works Camp Refugee Site
2020-06-09 16:41:12.802991 Shehu Musta Ii Refugee Site
2020-06-09 16:41:16.767680 Gssss Camp Bama Refugee Site
2020-06-09 16:41:21.883858 Wege Arrival Centre Refugee Site
2020-06-09 16:41:25.791826 Aisha Buhari Camp Refugee Site
2020-06-09 16:41:29.628963 Magistrate Refugee Site
2020-06-09 16:41:33.278837 Ggss Gajiganna Camp Refugee Site
2020-06-09 16:41:35.985744 Police Barracks Refugee site
2020-06-09 16:41:38.482277 Mobile Clinic Refugee site
2020-06-09 16:41:41.807267 Police Doki Refugee site
2020-06-09 16:41:44.718579 Upper Court Refugee site
2020-06-09 16:41:47.360158 Unity Camp Refugee site
2020-06-09 16:41:49.760003 Bulamatari Camp Refugee Site
2020-06-09 16:41:51.855226 Stadium Camp Refugee Site
2020-06-09 16:41:55.463631 Gardener Low Cost Refugee Site

In [52]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [578]:
import matplotlib.cm as cm
import matplotlib.colors as colors

import matplotlib as mpl

mpl.rcParams.update({'text.color':"white",
                     'axes.labelcolor':"white"})

def polar_plot(ax, bearings, n = 36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')
    
    norm = colors.Normalize(vmin=min(count)*0.8, vmax=max(count))
    cmap = cm.ScalarMappable(norm=norm, cmap=cm.jet)
    bc = [cmap.to_rgba(cl) for cl in count]

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2, 
                  color=bc, edgecolor='w', linewidth=0.1, alpha=1)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'DejaVu Sans', 'size':18, 'weight':'bold'}
    xtick_font = {'family':'DejaVu Sans', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'DejaVu Sans', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_facecolor('k')
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [179]:
filtered = dict((key, value) for key, value in bearings.items() if len(value) > 100)

In [442]:
print(len(bearings))
print(len(filtered))

291
192


In [443]:
camps['key'] = pd.Series(np.array(list(bearings.keys())), dtype = 'int')
camps.head()

,X,Y,name,cleaned_name,key
0,-6.865540,26.831305,Dakhla Refugee Camp,Dakhla,1.0
1,-8.010650,27.513829,Boujdour Refugee Camp,Boujdour,2.0
2,-8.023589,27.739910,El Aaiún Refugee Camp,El Aaiún,3.0
3,-8.720793,7.025102,Bahn Refugee Camp,Bahn,4.0
4,40.286872,0.190007,Dagahaley Refugee Camp,Dagahaley,5.0


In [444]:
keys = np.array(list(filtered.keys()))

In [424]:
plots = keys[1:100]

In [445]:
camps['cleaned_name'] = camps.name.str.replace("camp|refugee|settlement|reffugee|site", "", case = False)

In [446]:
camps.cleaned_name[72] = "Adi Harush"
camps.cleaned_name[14] = "Gondje"

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [427]:
reset = camps.set_index('key')

In [455]:
keys = np.array(list(filtered.keys()))

trimmed = reset.loc[keys, :]
trimmed.shape

(192, 4)

In [456]:
len(filtered) == len(trimmed)

True

In [457]:
n = 100
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, camp in zip(axes.flat, plots):
    polar_plot(ax, filtered[camp].dropna(), title=trimmed.cleaned_name[camp])

# add super title and save full image
suptitle_font = {'family':'DejaVu Sans', 'fontsize':60, 'fontweight':'normal', 'y':1.07}
fig.suptitle('Refugee Camp Network Orientation', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()

In [463]:
from scipy import stats

var = []

for key, value in filtered.items():
    var.append(stats.entropy(value))

In [465]:
trimmed['variance'] = var

In [504]:
omitna = trimmed.dropna()

In [505]:
ordered = omitna.sort_values('variance').index.to_numpy()

In [506]:
plots = ordered[0:100]

In [577]:
n = 100
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, camp in zip(axes.flat, plots):
    polar_plot(ax, filtered[camp].dropna(), title=reset.cleaned_name[camp])

# add super title and save full image
suptitle_font = {'family':'DejaVu Sans', 'fontsize':60, 'fontweight':'normal', 'y':1.07}
fig.suptitle('Refugee Camp Network Orientation', **suptitle_font)
fig.tight_layout()
fig.set_facecolor('k')
fig.subplots_adjust(hspace=0.35)
fig.savefig('street-orientations.png', facecolor=fig.get_facecolor(), dpi=120, bbox_inches='tight')
plt.close()